# Implementation timing 

This notebook uses the `%timeit` magic command to time different implementations in the artifact removal tool.
The notebook is set to use one file from the [Emotiv Epoc X](/Data/Emotiv%20Epoc%20X/) dataset. However, you can change the location and import section of the code, to test with other files.

In [9]:
#%% Import libraries
import mne
import sys
import numpy as np
sys.path.append("..") # Adds higher directory to python modules path.
from Functions import artifact_removal_tools as art

## Import and trim data

Select data to import and trim to a shorter length

In [10]:
# - Import data
full_path = 'C:\\Users\\danie\\OneDrive - University of Calgary\\PDF\\Proyects\\SSVEP_ART\\Data\\Emotiv Epoc X\\OKB\\SSVEP_17Hz_OKB_Mar22_EPOCX_126582_2021.03.22T17.45.11.06.00.edf'
edf_data = mne.io.read_raw_edf(full_path, verbose=False)

# - Organize data
srate = edf_data.info['sfreq']              # Sampling frequency [Hz]
ch_names = edf_data.info['ch_names']        # Channel names [list]
raw_data = edf_data.get_data(range(4,18))   # Raw data [channels x samples] [V]

# - Trim data
time_trim = [45, 55]  # Time start and end to trim EEG [sec]
time_vect = np.linspace(time_trim[0], time_trim[1], int(np.diff(time_trim)*srate))  # Time vector [sec]
raw_trim = raw_data[2,int(time_trim[0]*srate):int(time_trim[1]*srate)]              # Raw EEG trimmed [V]

## Time implementations

Switch implementations and time them. The implementations include the following options.
- SVD method
    - `sci` = Use scipy to calculate SVD
    - `np` = Use numpy to calculate SVD
- Antidiag method
    - `mask` = Compute matrix with antidiagonals and boolean mask, calculate mean of matrix
    - `simple` = Iterate through each antidiagonal and calculate mean value  

In [11]:
print('SVD_method = sci, antidiag_method = mask')
%timeit -r 5 art.remove_eyeblinks_cpu(raw_trim, srate, n_clusters=10, fd_threshold=1.6, window_length=0.3, svd_method='sci', antidiag_method='mask')

SVD_method = sci, antidiag_method = mask
110 ms ± 1.05 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [12]:
print('SVD_method = sci, antidiag_method = simple')
%timeit -r 5 art.remove_eyeblinks_cpu(raw_trim, srate, n_clusters=10, fd_threshold=1.6, window_length=0.3, svd_method='sci', antidiag_method='simple')

SVD_method = sci, antidiag_method = simple
121 ms ± 2.32 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [13]:
print('SVD_method = np, antidiag_method = mask')
%timeit -r 5 art.remove_eyeblinks_cpu(raw_trim, srate, n_clusters=10, fd_threshold=1.6, window_length=0.3, svd_method='np', antidiag_method='mask')

SVD_method = np, antidiag_method = mask
149 ms ± 2.38 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [14]:
print('SVD_method = np antidiag_method = mask')
%timeit -r 5 art.remove_eyeblinks_cpu(raw_trim, srate, n_clusters=10, fd_threshold=1.6, window_length=0.3, svd_method='np', antidiag_method='simple')

SVD_method = np antidiag_method = mask
181 ms ± 2.35 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


## Results

It seems from the results above that the fastest implementation is using `sci` and `mask` as the artifact removal tool options